In [ ]:
import os
from bb import *
import pandas as pd
import networkx as nx
import plotly.express as px
import matplotlib.pyplot as plt

### Plot NMR Graph

In [ ]:
def nmr_graph(fnmr):
    EdgeNMR.EID = 0
    nmr = NMR(fnmr)
    E = {edge.eid: edge for edge in nmr.pruneEdges}
    S = {s.sid: s for s in nmr.segments}    
    G = nx.Graph()
    s_lbl = lambda sid: '%d:%d' % (S[sid].i, S[sid].j)
    for sid in S:                
        G.add_node(s_lbl(sid), weight=S[sid].weight)
    
    for eid in E:
        e_lbl = 'E%d' % eid
        G.add_node(e_lbl)
        for sid in E[eid].sid:            
            G.add_edge(e_lbl, s_lbl(sid), weight=S[sid].weight)
    return G

def plot_graph(G):
    pos = nx.spring_layout(G)
    edge_labels = {(u,v):G[u][v]['weight'] for u,v in G.edges()}
    node_color = ['r' if node.startswith('E') else 'y' for node in G.nodes()]
    node_size = [300 if node.startswith('E') else 800 for node in G.nodes()]
    nx.draw_networkx_nodes(G, pos, 
        node_size=node_size, 
        node_shape='o', 
        node_color=node_color)
    nx.draw_networkx_labels(G, pos, font_color='k', font_size=8)
    nx.draw_networkx_edges(G, pos)
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=7)
    plt.show()

In [ ]:
G = nmr_graph('../DATA_TEST/testA.nmr')
plot_graph(G)

### READ LOG FILES

In [ ]:
WDIR = ['DATA_EPSD_00_DMAX_50', 'DATA_EPSD_00_DMAX_60']
df = []
for wdir in WDIR:
    dmax = int(wdir.split('_')[-1])
    for flog in sorted(os.listdir(wdir)):
        if not flog.endswith('.log'):
            continue
        pid = flog.replace('.log','')
        flog = os.path.join(wdir, flog)
        df_log = {'pid': pid, 'dmax':dmax, 'flog': flog}
        with open(flog, 'r') as fid:
            for row in fid:
                row = row.replace(':','').replace('\n','').split(' ')
                field = row[1]
                if 'fnmr' in field:
                    value = row[-1]
                elif 'time' in field:
                    value = float(row[-1])
                else:
                    value = int(row[-1])
                df_log[field] = value
        if ('costSBBU' in df_log) and (df_log['costSBBU'] == df_log['costRELAX']):
            df_log['costBB'] = df_log['costSBBU']
            df_log['timeBB'] = df_log['timeSBBU']
        df.append(df_log)
df = pd.DataFrame(df)
df['gap'] = (df['costSBBU'] - df['costRELAX']) / df['costRELAX']
df.head()

In [ ]:
df.groupby(by=['pid','dmax']).mean()

In [ ]:
df50 = df[df['dmax']==50]
df60 = df[df['dmax']==60]